Import packages

In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt

Specify name of approaches and dataset folders.

In [2]:
who = ['cresset_valid','schrodinger','amber','gaff','cgenff','consensus']
folders = ['bace', 'cdk2', 'jnk1', 'mcl1', 'ptp1b', 'p38', 'thrombin', 'tyk2']
mue_rmse = {}
kcal = 4.184 #kJ to kcal ratio

Adjust your OS and path to the directory here if you wish to rerun the notebook.

In [3]:
os_name = 'windows'
def set_wd(os_name):
    if os_name.lower() == 'linux':
        os.chdir('/home/max/')
    elif os_name.lower() == 'windows':
        os.chdir('C:/kuhn_et_al/wang_2015_datasets/')
    else:
        os.chdir('/some/path/to/directory')
def freenrgworkflow_init(base_folder, subfolder):
    results_dir = 'wang_2015_datasets/%s%s/' %(base_folder, subfolder)
    os.chdir(results_dir+'/results')

Get results of free energy calculations obtaining MUE per compound (MUE(c)).

In [4]:
def f_statistics(exp, com):
    F_of_R = 0.5*np.log((1+np.corrcoef(exp,com)[0,1])/(1-np.corrcoef(exp,com)[0,1]))
    var_of_F = 1/(len(exp)-3)
    F = (F_of_R - np.sqrt(var_of_F), F_of_R + np.sqrt(var_of_F))
    R_of_F = ((np.e**(2*F[0])-1)/(np.e**(2*F[0])+1), (np.e**(2*F[1])-1)/(np.e**(2*F[1])+1))
    ci = (R_of_F[0]**2, R_of_F[1]**2)
    t99 = stats.t.ppf(0.99, len(exp)-2)
    Rcrit = (t99/np.sqrt(len(exp)+t99**2))**2
    return(ci, Rcrit)

In [5]:
def analyze_results(who, which, n_bootstrap, gen_plot):
    if n_bootstrap < 1000:
        raise ValueError('Please set n_bootstrap to at least 1000 iterations.')
    
    print('Calculating results for %s from %s.' %(which.upper(), who.upper()))
    set_wd(os_name)
    
    #Get results
    affs = pd.read_csv('wang_2015_datasets/final_results_%s.csv' %which)
    affs = affs.drop(len(affs)-1) # to drop the "sum" row
    exp = affs.loc[:,'dGexptl']
    com = affs.loc[:,who]
    mue = abs(com-exp)
    rmse = mue**2
    fstat,R_ref = f_statistics(exp, com)
    
    #generate plot
    if gen_plot:
        plt.scatter(exp,com)
        plt.grid()
    
    #Bootstrapping
    boot_affs = []
    boot_mues = []
    boot_rmses = []
    boot_taus = []
    for _ in range(n_bootstrap):
        exps = []
        coms = []
        mues = []
        rmses = []
        for i in range(len(affs)-1):
            r = random.randint(0,len(exp)-1)
            exps.append(exp[r])
            coms.append(com[r])
            mues.append(mue[r])
            rmses.append(rmse[r])
        boot_affs.append(np.corrcoef(exps,coms))
        boot_mues.append(np.mean(mues))
        boot_rmses.append(np.sqrt(sum(rmses)/len(rmses)))
        boot_taus.append(stats.kendalltau(exps,coms))
    bootaffs = (np.mean(boot_affs), np.std(boot_affs))
    bootmues = (np.mean(boot_mues), np.std(boot_mues))
    bootrmses = (np.mean(boot_rmses), np.std(boot_rmses))
    boottaus = (np.mean(boot_taus), np.std(boot_taus))
    
    #Print Results
    print('Number of ligands: %s' %len(exp))
    print('R: %.2f +/- %.2f' %(np.corrcoef(exp,com)[0,1],bootaffs[1]))
    print('R²: %.2f' %np.corrcoef(exp,com)[0,1]**2)
    print('CI_0.95 for R²: [%.2f - %.2f]' %fstat)
    print('critical R² for p=0.99: %.2f' %R_ref)
    print('MUE(l): %.2f +/- %.2f' %(np.mean(mue), bootmues[1]))
    print('RMSE(l): %.2f +/- %.2f' %(np.sqrt(np.mean(rmse)), bootrmses[1]))
    print('Kendall tau: %.2f +/- %.2f' %(stats.kendalltau(exp,com)[0], boottaus[1]))
    print('-'*52)

Calcualte the error per perturbation (MUE(p)).

In [6]:
def error_per_perturbation(who, folders, n_bootstrap, info): #True for info
    if n_bootstrap < 1000:
        raise ValueError('Please set n_bootstrap to at least 1000 iterations.')
    
    for folder in folders:
        print(who.upper(), '-', folder.upper())
        dataset_name = folder
        set_wd(os_name)
        freenrgworkflow_init(dataset_name, '')
        tmp = errors_for_dataset(who, info) 
        errors = single_topology_errors(tmp[0], dataset_name.rsplit('_')[-1], tmp[1], tmp[2], info)
        print('Number of calculations considered:', len(errors))
        
        #Bootstrapping
        boot_means = []
        boot_rmses = []
        for _ in range(n_bootstrap):
            bootsample = np.random.choice(errors, size = len(errors), replace=True)
            boot_means.append(bootsample.mean())
            boot_rmses.append(np.sqrt(sum(bootsample**2)/len(bootsample)))
        bootmean_std = np.std(boot_means)
        bootrmse_std = np.std(boot_rmses)
       
        #Print results
        print('MUE(p): %.2f +/- %.2f kcal/mol, %.2f +/- %.2f kJ/mol'
              %(np.mean(errors), bootmean_std, np.mean(errors)*kcal, bootmean_std*kcal))
        print('RMSE(p): %.2f +/- %.2f, kcal/mol %.2f +/- %.2f kJ/mol'
              %(np.sqrt(sum(errors**2)/len(errors)), bootrmse_std,
                np.sqrt(sum(errors**2)/len(errors))*kcal, bootrmse_std*kcal))
        print('-'*53)
        mue_rmse[folder] = [folder, 'MUE(p): %.2f +/- %.2f kcal/mol, %.2f +/- %.2f kJ/mol'
                            %(np.mean(errors), bootmean_std, np.mean(errors)*kcal, bootmean_std*kcal),
                           'RMSE(p): %.2f +/- %.2f, kcal/mol %.2f +/- %.2f kJ/mol'
                          %(np.sqrt(sum(errors**2)/len(errors)), bootrmse_std,
                            np.sqrt(sum(errors**2)/len(errors))*kcal, bootrmse_std*kcal)]
    
    #Print summary
    print('Summary for %s:' %who.upper())
    [print(mue_rmse[res][0].ljust(8),'-',mue_rmse[res][1],mue_rmse[res][2]) for res in mue_rmse.keys()]
    print('='*117,'\n')

In [7]:
def errors_for_dataset(who, info):
    exp = {}
    com = []
    errors = []
    no_exp = 0
    
    #convert kJ to kcal if dataset from Gapsys et al., 2020
    if who in ['gaff', 'cgenff', 'consensus']:
        kcal = 4.184
    else:
        kcal = 1.000
    
    #read the experimental affinities
    with open ('experimental_dG.csv', 'r') as f:
        lines = f.readlines()
        for line in lines:
            if line[0] != '#':
                exp[line.split(',')[0]] = float(line.split(',')[1])
    if info:
        print('Experimental affinities from mulit-sdf file of Wang et al., 2015:')
        [print(e,exp[e]) for e in exp.keys()]
    
    #read results from computations
    if who == 'cresset_valid':
        com_both = []
        with open ('summary/fep_results_freenrgworkflow_cresset_valid.csv', 'r') as f:
            lines = f.readlines()
            [com_both.append(line) for line in lines if not '#' in line]
            for i in range(len(com_both)):
                if i %2 == 0:
                    fw = float(com_both[i].split(',')[2])
                    bw = float(com_both[i+1].split(',')[2])
                    fw_err = float(com_both[i].split(',')[3])
                    bw_err = float(com_both[i+1].split(',')[3])
                    com.append('%s,%s,%s,%s' %(com_both[i].split(',')[0], com_both[i].split(',')[1], \
                                               round((fw-bw)/2,3), round((fw_err+bw_err)/2,3)))
    else:
        with open ('summary/fep_results_freenrgworkflow_%s.csv' %who, 'r') as f:
            lines = f.readlines()
            [com.append(line) for line in lines if not '#' in line]
    if info:
        print('Computed affnities according to %s.' %who)
        [print(c) for c in com]
    
    #calculate difference between experimental and computed values
    if info:
        print('-'*23,'\nlig1,lig2,com,exp,error\n','-'*21)
    for entry in com:
        try:
            A,B,co = entry.split(',')[0:3]
            co = float(co)/kcal
            ex = round(exp[B]-exp[A],3)
            if info:
                print(A,B,round(co,2),ex,round(abs(co-ex),2))
            errors.append(round(abs(co-ex),3))
        except:
            no_exp += 1
            if info:
                print('No experimental ddG for:', entry.strip('\n').strip(','))
            pass
    if info:
        print(errors)
    errors = np.array(errors)
    
    #print results
    if info:
        print('Number of calculations considered:', len(errors))
        print('Missing experimental affinities: %s perturbations' %no_exp)
        print('MUE (p):', round(np.mean(errors),4))
        print('RMSE (p):', round(np.sqrt(sum(errors**2)/len(errors)),4))
    
    #return results
    return(list(errors), exp, com)

In [8]:
#Function for generating ligand pairs using intermediate ligands (only used in the Cresset validation dataset).
def single_topology_errors(list_of_errors, dataset, experimental, computed, info):
    err_add = list_of_errors
    err_dict = {}
    if info:
        print('Checking for further perturbations.')
        
    #Check whether and which additional perturbations exists
    if dataset not in ['bace', 'p38', 'ptp', 'tyk']:
        triples = []
        errors = []
        if info:
            print('No further perturbations found.')
    elif dataset == 'tyk':
        triples = [('ejm31', 'jmc28', 'jul01'), ('ejm31', 'ejm49', 'jul01'), ('ejm31', 'ejm48', 'jul01'), \
                   ('ejm31', 'ejm47', 'jul01'), ('ejm31', 'ejm46', 'jul01')]
    elif dataset == 'p38':
        triples = [('3fly', '2j', 'mk3'), ('3fly', '3fln', 'mk3'), ('2c', '3flz', 'mk1'), ('2c', '2f', 'mk1'), \
                   ('2x', '2j', 'mk3'), ('2u', '2k', 'mk2'), ('2q', '2k', 'mk2'), ('3fly', '2v', 'mk4'), \
                   ('2y', '2v', 'mk4'), ('2bb', '2v', 'mk4'), ('3fmh', '2v', 'mk4'), ('3fmk', '2v', 'mk4')]
    elif dataset == 'ptp':
        triples = [('23466', '23471', 'MK2'), ('23466', '23468', 'MK2'), ('23466', '20669', 'MK1'), \
                   ('23466', '23474', 'MK2')]
    elif dataset == 'bace':
        triples = [( '4b',  '4p', 'mk3'), ( '4d',  '4p', 'mk3'), ('13k',  '4p', 'mk3'), ('13m',  '4p', 'mk3'), \
                   ('13j',  '4p', 'mk3'), ('13n',  '4p', 'mk3'), ('13e',  '4p', 'mk2'), ('13f',  '4p', 'mk2'), \
                   ('13g',  '4p', 'mk2'), ('13e', '13d', 'mk1'), ('13f', '13d', 'mk1'), ('13g', '13d', 'mk1'), \
                   ('13o', '13d', 'mk1'), ('13a', '13d', 'mk1'), ('13i', '13d', 'mk1'), ( '4p', '17h', 'mk2')]
    
    #generate affinity pairs for A <=> B by combining A <=> I and I <=> B 
    for triple in triples:
        counter = 1
        for entry in computed:
            entry = entry.split(',')
            if entry[0] == triple[2]:
                if entry[1] == triple[0]:
                    counter += 1
                    step1 = -float(entry[2])
                if entry[1] == triple[1]:
                    counter += 1
                    step2 = float(entry[2])
            if entry[1] == triple[2]:
                if entry[0] == triple[0]:
                    counter += 1
                    step1 = float(entry[2])
                if entry[0] == triple[1]:
                    counter += 1
                    step2 = -float(entry[2])
            if counter %3 == 0:
                counter += 1
                exp = round(experimental[triple[1]] - experimental[triple[0]],3)
                err_dict['%s_to_%s' %(triple[0],triple[1])] = (round(step1+step2,3), exp)
                if info:
                    print('Found: %s_to_%s' %(triple[0],triple[1]), (round(step1+step2,3), exp))
                    
    #append the generated affinity pairs to the error dictionary
    for key in err_dict.keys():
        err_add.append(round(abs(err_dict[key][0] - err_dict[key][1]),3))
    errors = np.array(err_add)
    return(errors)

Run part 1 (Results and MUE(c)).

In [9]:
for i in who:
    for f in folders:
        analyze_results(i,f,10000, False) #who,which,n_bootstrap,generate_plot
    print('='*52,'\n')

Calculating results for BACE from CRESSET_VALID.
Number of ligands: 36
R: 0.44 +/- 0.30
R²: 0.19
CI_0.95 for R²: [0.08 - 0.32]
critical R² for p=0.99: 0.14
MUE(l): 0.76 +/- 0.09
RMSE(l): 0.94 +/- 0.10
Kendall tau: 0.28 +/- 0.17
----------------------------------------------------
Calculating results for CDK2 from CRESSET_VALID.
Number of ligands: 16
R: 0.74 +/- 0.15
R²: 0.54
CI_0.95 for R²: [0.34 - 0.71]
critical R² for p=0.99: 0.30
MUE(l): 0.69 +/- 0.14
RMSE(l): 0.87 +/- 0.15
Kendall tau: 0.43 +/- 0.26
----------------------------------------------------
Calculating results for JNK1 from CRESSET_VALID.
Number of ligands: 21
R: 0.72 +/- 0.15
R²: 0.52
CI_0.95 for R²: [0.35 - 0.67]
critical R² for p=0.99: 0.23
MUE(l): 0.76 +/- 0.13
RMSE(l): 0.96 +/- 0.15
Kendall tau: 0.55 +/- 0.28
----------------------------------------------------
Calculating results for MCL1 from CRESSET_VALID.
Number of ligands: 42
R: 0.74 +/- 0.14
R²: 0.55
CI_0.95 for R²: [0.43 - 0.65]
critical R² for p=0.99: 0.12
M

Number of ligands: 34
R: 0.68 +/- 0.17
R²: 0.46
CI_0.95 for R²: [0.33 - 0.59]
critical R² for p=0.99: 0.15
MUE(l): 0.62 +/- 0.08
RMSE(l): 0.76 +/- 0.08
Kendall tau: 0.50 +/- 0.26
----------------------------------------------------
Calculating results for THROMBIN from GAFF.
Number of ligands: 11
R: 0.24 +/- 0.43
R²: 0.06
CI_0.95 for R²: [0.01 - 0.29]
critical R² for p=0.99: 0.42
MUE(l): 0.66 +/- 0.11
RMSE(l): 0.75 +/- 0.11
Kendall tau: 0.24 +/- 0.28
----------------------------------------------------
Calculating results for TYK2 from GAFF.
Number of ligands: 16
R: 0.69 +/- 0.21
R²: 0.47
CI_0.95 for R²: [0.26 - 0.65]
critical R² for p=0.99: 0.30
MUE(l): 0.87 +/- 0.11
RMSE(l): 0.97 +/- 0.11
Kendall tau: 0.41 +/- 0.24
----------------------------------------------------

Calculating results for BACE from CGENFF.
Number of ligands: 36
R: 0.56 +/- 0.25
R²: 0.32
CI_0.95 for R²: [0.19 - 0.45]
critical R² for p=0.99: 0.14
MUE(l): 0.75 +/- 0.12
RMSE(l): 1.02 +/- 0.15
Kendall tau: 0.38 +/- 0.2

Run part 2 (MUE(p)).

In [10]:
for i in who:
    error_per_perturbation(i, folders, 10000, False) #who, folders, n_bootstrap, info/verbose

CRESSET_VALID - BACE
Number of calculations considered: 55
MUE(p): 0.95 +/- 0.10 kcal/mol, 3.96 +/- 0.41 kJ/mol
RMSE(p): 1.20 +/- 0.12, kcal/mol 5.01 +/- 0.50 kJ/mol
-----------------------------------------------------
CRESSET_VALID - CDK2
Number of calculations considered: 22
MUE(p): 0.95 +/- 0.12 kcal/mol, 3.98 +/- 0.50 kJ/mol
RMSE(p): 1.11 +/- 0.12, kcal/mol 4.65 +/- 0.49 kJ/mol
-----------------------------------------------------
CRESSET_VALID - JNK1
Number of calculations considered: 34
MUE(p): 0.77 +/- 0.10 kcal/mol, 3.24 +/- 0.42 kJ/mol
RMSE(p): 0.97 +/- 0.11, kcal/mol 4.08 +/- 0.47 kJ/mol
-----------------------------------------------------
CRESSET_VALID - MCL1
Number of calculations considered: 64
MUE(p): 1.36 +/- 0.12 kcal/mol, 5.71 +/- 0.52 kJ/mol
RMSE(p): 1.69 +/- 0.16, kcal/mol 7.09 +/- 0.67 kJ/mol
-----------------------------------------------------
CRESSET_VALID - PTP1B
Number of calculations considered: 26
MUE(p): 1.09 +/- 0.18 kcal/mol, 4.58 +/- 0.76 kJ/mol
RMSE(p)

MUE(p): 0.84 +/- 0.09 kcal/mol, 3.51 +/- 0.39 kJ/mol
RMSE(p): 1.09 +/- 0.10, kcal/mol 4.58 +/- 0.44 kJ/mol
-----------------------------------------------------
GAFF - CDK2
Number of calculations considered: 25
MUE(p): 0.68 +/- 0.12 kcal/mol, 2.86 +/- 0.51 kJ/mol
RMSE(p): 0.92 +/- 0.15, kcal/mol 3.84 +/- 0.62 kJ/mol
-----------------------------------------------------
GAFF - JNK1
Number of calculations considered: 31
MUE(p): 0.80 +/- 0.12 kcal/mol, 3.36 +/- 0.51 kJ/mol
RMSE(p): 1.05 +/- 0.16, kcal/mol 4.41 +/- 0.68 kJ/mol
-----------------------------------------------------
GAFF - MCL1
Number of calculations considered: 71
MUE(p): 1.23 +/- 0.11 kcal/mol, 5.17 +/- 0.48 kJ/mol
RMSE(p): 1.56 +/- 0.13, kcal/mol 6.54 +/- 0.54 kJ/mol
-----------------------------------------------------
GAFF - PTP1B
Number of calculations considered: 49
MUE(p): 0.90 +/- 0.10 kcal/mol, 3.76 +/- 0.40 kJ/mol
RMSE(p): 1.12 +/- 0.11, kcal/mol 4.67 +/- 0.47 kJ/mol
------------------------------------------------